### Por realizar:

* Limpiar data
* Verificar sintomas y outcome.
* Cuanta gente existe por grupo etario
* Mostrar sintomás que más se repite
* Ver modelo de regresión linear simple para aplicar acá


## Análisis de datos COVID-19

### Objetivo:
#### Identificación de individuos que presentan síntomas y características específicas, padecen la enfermedad COVID-19 y están con riesgo de muerte.

#### Se utilizarán herramientas de análisis de datos con Python, Pandas, Matplotlib, numpy y scikit-learn.

In [4]:
# Importamos las librerías necesarias para el análisis de datos.
# Pandas para el manejo y tratamiento de los datos
# Numpy para poder trabajar con matrices multidimensionales
# Matplotlib para visualización de data
# Seaborn para visualización de data, basado en matplotlib
# Re para expresiones regulares

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import re

# Se indica a matplotlib mostrar el gráfico en la celda a ejecutar y guarda el gráfico en el notebook.
%matplotlib inline

# Otros
import glob, os


### 0) Preparación de la información y data
#### Importaremos el dataset 

In [5]:
# Importamos el archivo CSV como un DF
df_full = pd.read_csv('latestdata.csv')


/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1,2,9,10,12,13,14,15,16,17,19,21,22,23,24,25,26,27,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# Creamos un nuevo DF con las filas donde las edades no son nulos
df_preProcessing = df_full[df_full['age'].notna()]


In [7]:
# Creamos un nuevo DF solo con las columnas que nos sirven y fueron previamente limpiadas por edad.
df_clean = df_preProcessing[[
     'age', 'sex', 'date_onset_symptoms', 
     'date_admission_hospital', 'date_confirmation', 
     'symptoms', 'chronic_disease_binary', 'chronic_disease', 
     'source', 'outcome', 'date_death_or_discharge', 'notes_for_discussion' 
     ]]


In [8]:
# Analizamos las filas con datos nulos que quedan en el dataset, la edad es 0 ya que eliminamos todos los registros con edad null
obj = df_clean.isnull().sum()
for key, value in obj.iteritems():
    print(key, ",", value)

age , 0
sex , 1756
date_onset_symptoms , 78534
date_admission_hospital , 240989
date_confirmation , 218
symptoms , 241499
chronic_disease_binary , 0
chronic_disease , 242915
source , 154102
outcome , 240552
date_death_or_discharge , 242588
notes_for_discussion , 242827


### 1) Análisis, limpieza y agrupación de datos por Edad


In [9]:
# Cuenta total de registros por edad
ageTotalCount = df_clean.age.count()
print("Existen {} registros de edad".format(ageTotalCount))


Existen 243077 registros de edad


In [10]:
# Visualización de formatos de edad
# df_clean.age.unique()
print("ver edad")

ver edad


In [70]:
# Visualización del DF sin nulos en la edad
df_clean.head()


,age,sex,date_onset_symptoms,date_admission_hospital,date_confirmation,symptoms,chronic_disease_binary,chronic_disease,source,outcome,date_death_or_discharge,notes_for_discussion
1,78,male,NaN,NaN,21.02.2020,NaN,False,NaN,https://www.corriere.it/cronache/20_febbraio_2...,death,22.02.2020,NaN
2,61,female,NaN,NaN,14.02.2020,NaN,False,NaN,https://www.channelnewsasia.com/news/singapore...,discharge,17.02.2020,NaN
27,66,male,NaN,NaN,07.02.2020,NaN,False,NaN,https://m.weibo.cn/detail/4469604653292027,NaN,NaN,https://www.google.com/maps/place/Zhenlai+Coun...
28,27,female,NaN,NaN,07.02.2020,NaN,False,NaN,https://m.weibo.cn/detail/4469604653292027,NaN,NaN,NaN
29,17,male,NaN,NaN,07.02.2020,NaN,False,NaN,https://m.weibo.cn/detail/4469604653292027,NaN,NaN,NaN


In [79]:
# Copia del DF sin nulos para modificarlo y poder trabajar las edades
df_test = df_clean

df_test.count()

age                        243077
sex                        241321
date_onset_symptoms        164543
date_admission_hospital      2088
date_confirmation          242859
symptoms                     1578
chronic_disease_binary     243077
chronic_disease               162
source                      88975
outcome                      2525
date_death_or_discharge       489
notes_for_discussion          250
dtype: int64

### Implementar Regex para estandarizar la data
#### Usar RegEX para validar los siguientes casos:

* Numeros enteros, guardar como enteros         ^\d\d?$
* Rango de edades, obtener la media y guardar   ^\d\d?[-]\d\d?$
* Numeros decimales, redondear y guardar        ^\d(\.\d+)?$
* Edades con signo + / -                        ^\d\d?[-]?[+]?$
* Numeros y letras, obtener número y guardar    ^\d\d?\s\w+?$

RegEx devuelve un arreglo con el valor que haga match con la expresión

In [86]:
# Declaración de expresiones regulares
ageInt = '^\d\d?$'
ageRange = '^\d\d?[-]\d\d?$'
ageFloat = '^\d(\.\d+)?$'
ageSign = '^\d\d?[-]?[+]?$'
ageText = '^\d\d?\s\w+?$'

# Arreglo para guardar nueva lista de edades
new_ages = []

# Recorremos las edades para estandarizar la información
for ages in df_test.age:
    strAges = str(ages)
    # RegEx Numeros enteros
    if re.findall(ageInt, strAges):
        new_ages.append(ages)
    # RegEx Rango de numeros
    elif re.findall(ageRange, strAges):
        indexes = ages.split('-')
        intIndexes = list(map(int, indexes))
        medianIndex = np.median(intIndexes)
        new_ages.append(int(medianIndex))
    # RegEx Numeros flotantes
    elif re.findall(ageFloat, strAges):
        formattedAge = int(round(float(ages)))
        new_ages.append(formattedAge)
    # ReEx Edades con signos (+/-)
    elif re.findall(ageSign, strAges):
        ageSt = re.findall(r'\d+', ages)
        new_ages.append(int(ageSt[0]))
    # Regex Edades con texto
    elif re.findall(ageText, strAges):
        monthAge = ages.split()
        monthConversion = int(monthAge[0]) / 12
        ageConverted = int(round(monthConversion))
        new_ages.append(ageConverted)
    else:
        new_ages.append("ERROR")
        
    
# Agregamos una nueva columna al DF con la edades nuevas
df_test["new_age"] = new_ages

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [88]:
print(df_test.head())

       age     sex date_onset_symptoms date_admission_hospital  \
1       78    male                 NaN                     NaN   
2       61  female                 NaN                     NaN   
27      66    male                 NaN                     NaN   
28      27  female                 NaN                     NaN   
29      17    male                 NaN                     NaN   
30      51  female                 NaN                     NaN   
31      68    male                 NaN                     NaN   
32      26    male                 NaN                     NaN   
33      30    male                 NaN                     NaN   
34      53  female                 NaN                     NaN   
35      35    male                 NaN                     NaN   
37      27  female                 NaN                     NaN   
38      28    male                 NaN                     NaN   
39      58  female                 NaN                     NaN   
40      33

In [89]:
# Export DF to excel
# df_test.to_excel("reAged.xlsx")  